In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
#import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import random
from shutil import copyfile
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
 #   for filename in filenames:
  #      print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [2]:
base_url = "/kaggle/input/cell-images-for-detecting-malaria/cell_images/"

print(len(os.listdir(base_url + "Uninfected")))
print(len(os.listdir(base_url + "Parasitized")))

13780
13780


In [3]:
try:
    os.mkdir("/kaggle/images/")
    os.mkdir("/kaggle/images/train/")
    os.mkdir("/kaggle/images/val/")
    os.mkdir("/kaggle/images/train/Infected/")
    os.mkdir("/kaggle/images/val//Infected/")
    os.mkdir("/kaggle/images/train/Uninfected/")
    os.mkdir("/kaggle/images/val/Uninfected/")
except OSError as e:
    print(e)

In [4]:
TRAIN_DIR = "/kaggle/images/train/"
VAL_DIR = "/kaggle/images/val/"

TRAIN_INFECTED_DIR = "/kaggle/images/train/Infected/"
TRAIN_UNINFECTED_DIR = "/kaggle/images/train/Uninfected/"
VAL_INFECTED_DIR = "/kaggle/images/val/Infected/"
VAL_UNINFECTED_DIR = "/kaggle/images/val/Uninfected/"

In [5]:
train_split = 0.9
size = len(os.listdir(base_url + "Uninfected"))

In [6]:
train_size = int(size*train_split)

print(train_size)
print(size - train_size)

12402
1378


In [7]:
list_Uninfected = random.sample( os.listdir(base_url + "Uninfected"), size)
list_Infected = random.sample( os.listdir(base_url + "Parasitized"), size)

print(len(list_Uninfected))
print(len(list_Infected))

13780
13780


In [8]:
for  i in range(size):
    if i < train_size:
        copyfile(base_url + "Uninfected/" + list_Uninfected[i], TRAIN_UNINFECTED_DIR + list_Uninfected[i])
        copyfile(base_url + "Parasitized/" + list_Infected[i], TRAIN_INFECTED_DIR + list_Infected[i])
    else:
        copyfile(base_url + "Uninfected/" + list_Uninfected[i], VAL_UNINFECTED_DIR + list_Uninfected[i])
        copyfile(base_url + "Parasitized/" + list_Infected[i], VAL_INFECTED_DIR + list_Infected[i])

In [9]:
print(len(os.listdir(TRAIN_UNINFECTED_DIR)))
print(len(os.listdir(TRAIN_INFECTED_DIR)))
print(len(os.listdir(VAL_UNINFECTED_DIR)))
print(len(os.listdir(VAL_INFECTED_DIR)))

12402
12402
1378
1378


In [10]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
                                   #fill_mode = 'nearest')
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size = (100, 100),
    #batch_size = 128,
    class_mode = 'binary')

validation_datagen = ImageDataGenerator(rescale = 1./255)
validation_generator = validation_datagen.flow_from_directory(
    VAL_DIR,
    target_size = (100, 100),
    #batch_size = 16,
    class_mode = 'binary')

Found 24802 images belonging to 2 classes.
Found 2756 images belonging to 2 classes.


In [11]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape = (100, 100, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape = (100, 100, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation = "relu"),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation="relu"),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(256, (3, 3), activation="relu"),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation="relu"),
    #tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

model.compile(loss = "binary_crossentropy", optimizer='adam', metrics=['acc'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 98, 98, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 49, 49, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 47, 47, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 23, 23, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 21, 21, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 10, 10, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 8, 128)         7

In [12]:
history = model.fit_generator(train_generator,
                              epochs=15,
                              verbose=1,
                              validation_data=validation_generator)

Train for 776 steps, validate for 87 steps
Epoch 1/15
776/776 [==============================] - 87s 112ms/step - loss: 0.5105 - acc: 0.6943 - val_loss: 0.1656 - val_acc: 0.9543
Epoch 2/15
776/776 [==============================] - 82s 105ms/step - loss: 0.2459 - acc: 0.9149 - val_loss: 0.1298 - val_acc: 0.9572
Epoch 3/15
776/776 [==============================] - 82s 105ms/step - loss: 0.2330 - acc: 0.9196 - val_loss: 0.1430 - val_acc: 0.9579
Epoch 4/15
776/776 [==============================] - 82s 105ms/step - loss: 0.2205 - acc: 0.9232 - val_loss: 0.1453 - val_acc: 0.9546
Epoch 5/15
776/776 [==============================] - 81s 104ms/step - loss: 0.2171 - acc: 0.9251 - val_loss: 0.1325 - val_acc: 0.9565
Epoch 6/15
776/776 [==============================] - 83s 106ms/step - loss: 0.2106 - acc: 0.9256 - val_loss: 0.1285 - val_acc: 0.9528
Epoch 7/15
776/776 [==============================] - 82s 105ms/step - loss: 0.2100 - acc: 0.9246 - val_loss: 0.1257 - val_acc: 0.9568
Epoch 8/15
7

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt


acc=history.history['acc']
val_acc=history.history['val_acc']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(len(acc)) # Get number of epochs

#------------------------------------------------
# Plot training and validation accuracy per epoch
#------------------------------------------------
plt.plot(epochs, acc, 'r', "Training Accuracy")
plt.plot(epochs, val_acc, 'b', "Validation Accuracy")
plt.title('Training and validation accuracy')
plt.figure()

#------------------------------------------------
# Plot training and validation loss per epoch
#------------------------------------------------
plt.plot(epochs, loss, 'r', "Training Loss")
plt.plot(epochs, val_loss, 'b', "Validation Loss")


plt.title('Training and validation loss')